In [1]:
#import dependencies
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st
import os

In [2]:
# Read single year crime files and store into final data frame
dunkin_csv = "final_dunkin_loc.csv"
mcdonalds_csv = "final_mcd_loc.csv"
panera_csv = "final_panera_loc.csv"
starbucks_csv = "sbux_loc.csv"

# pd.read_csv('file.csv', index_col=0)
dunkin_df = pd.read_csv(dunkin_csv, index_col=0).reset_index().rename(columns= {'id':'address'}).drop(['strLocation'],axis=1)
mcdonalds_df = pd.read_csv(mcdonalds_csv, index_col=0).reset_index().rename(columns= {'id':'address'}).drop(['strLocation'],axis=1)
panera_df = pd.read_csv(panera_csv, index_col=0).reset_index().rename(columns= {'id':'address'}).drop(['strLocation'],axis=1)
starbucks_df = pd.read_csv(starbucks_csv, index_col=0).reset_index().rename(columns={'strLocation':'address'}).drop(['id'],axis=1)
starbucks_df

,address,latitude,longitude,city,state,zip,Store
0,32nd & Range Line,37.052427,-94.478940,Joplin,MO,64804,Starbucks
1,Target Joplin T-774,37.085213,-94.473855,Joplin,MO,64801,Starbucks
2,3rd & Range Line,37.088028,-94.477036,Joplin,MO,64801,Starbucks
3,St. Louis Premium Outlets,38.644100,-90.631922,Chesterfield,MO,63005,Starbucks
4,Taylor & Manchester,38.582168,-90.628030,Wildwood,MO,63040,Starbucks
...,...,...,...,...,...,...,...
84,157 & Center Grove Rd,38.787750,-89.978912,Edwardsville,IL,62025,Starbucks
85,Troy & Governor's Pkwy,38.788790,-89.951888,Edwardsville,IL,62025,Starbucks
86,TARGET EDWARDSVILLE T-733,38.782621,-89.949238,Edwardsville,IL,62025,Starbucks
87,Target Belleville T-2330,38.511200,-90.014600,Belleville,IL,62226,Starbucks


In [3]:
#Combine all csv files
frames = [dunkin_df, mcdonalds_df, panera_df, starbucks_df]
combine_df = pd.concat(frames)
combine_df

,address,latitude,longitude,city,state,zip,Store
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,Independence,MO,64055,Dunkin'
1,2400 Missouri 291 MO 64057,39.075001,-94.379703,Independence,MO,64055,Dunkin'
2,2400 Missouri 291 MO 64057,39.075001,-94.379703,Independence,MO,64055,Dunkin'
3,2400 Missouri 291 MO 64057,39.075001,-94.379703,Independence,MO,64055,Dunkin'
4,33 E Lake St IL 60101,41.931216,-87.987127,Addison,IL,60101,Dunkin'
...,...,...,...,...,...,...,...
84,157 & Center Grove Rd,38.787750,-89.978912,Edwardsville,IL,62025,Starbucks
85,Troy & Governor's Pkwy,38.788790,-89.951888,Edwardsville,IL,62025,Starbucks
86,TARGET EDWARDSVILLE T-733,38.782621,-89.949238,Edwardsville,IL,62025,Starbucks
87,Target Belleville T-2330,38.511200,-90.014600,Belleville,IL,62226,Starbucks


In [4]:
combine_df.drop_duplicates(subset=None, keep='first', inplace=True)
combine_df

,address,latitude,longitude,city,state,zip,Store
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,Independence,MO,64055,Dunkin'
4,33 E Lake St IL 60101,41.931216,-87.987127,Addison,IL,60101,Dunkin'
5,1750 W Lake St IL 60101,41.950451,-88.030146,Addison,IL,60101,Dunkin'
8,1319 East Algonquin Road IL 60102,42.161334,-88.276569,Algonquin,IL,60102,Dunkin'
9,2543 W Algonquin Rd IL 60102,42.175207,-88.338530,Algonquin,IL,60102,Dunkin'
...,...,...,...,...,...,...,...
84,157 & Center Grove Rd,38.787750,-89.978912,Edwardsville,IL,62025,Starbucks
85,Troy & Governor's Pkwy,38.788790,-89.951888,Edwardsville,IL,62025,Starbucks
86,TARGET EDWARDSVILLE T-733,38.782621,-89.949238,Edwardsville,IL,62025,Starbucks
87,Target Belleville T-2330,38.511200,-90.014600,Belleville,IL,62226,Starbucks


In [5]:
combine_df = combine_df.loc[combine_df['state']=='MO']
combine_df

,address,latitude,longitude,city,state,zip,Store
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,Independence,MO,64055,Dunkin'
0,3996 Jeffco Blvd MO 63010,38.405690,-90.376802,Arnold,MO,63010,McDonald's
1,130 Arnold Crossroads Ctr MO 63010,38.445494,-90.374454,Arnold,MO,63010,McDonald's
2,2201 Michigan Ave MO 63010,38.437040,-90.377478,Arnold,MO,63010,McDonald's
3,2005 Richardson Rd MO 63010,38.418379,-90.390784,Arnold,MO,63010,McDonald's
...,...,...,...,...,...,...,...
78,S Jefferson and Walnut,38.629806,-90.216753,St. Louis,MO,63103,Starbucks
79,Marriott St. Louis Grand,38.630655,-90.191852,St. Louis,MO,63101,Starbucks
80,Ballpark Village,38.624702,-90.193569,St. Louis,MO,63102,Starbucks
81,St. Louis Hilton at the Ballpark,38.625172,-90.190579,St. Louis,MO,63102,Starbucks


In [6]:
combine_df['index_col']=combine_df.index
combine_df

C:\Users\kelly\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,address,latitude,longitude,city,state,zip,Store,index_col
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,Independence,MO,64055,Dunkin',0
0,3996 Jeffco Blvd MO 63010,38.405690,-90.376802,Arnold,MO,63010,McDonald's,0
1,130 Arnold Crossroads Ctr MO 63010,38.445494,-90.374454,Arnold,MO,63010,McDonald's,1
2,2201 Michigan Ave MO 63010,38.437040,-90.377478,Arnold,MO,63010,McDonald's,2
3,2005 Richardson Rd MO 63010,38.418379,-90.390784,Arnold,MO,63010,McDonald's,3
...,...,...,...,...,...,...,...,...
78,S Jefferson and Walnut,38.629806,-90.216753,St. Louis,MO,63103,Starbucks,78
79,Marriott St. Louis Grand,38.630655,-90.191852,St. Louis,MO,63101,Starbucks,79
80,Ballpark Village,38.624702,-90.193569,St. Louis,MO,63102,Starbucks,80
81,St. Louis Hilton at the Ballpark,38.625172,-90.190579,St. Louis,MO,63102,Starbucks,81


In [7]:
combine_df[["latitude", "index_col"]]=combine_df[["latitude", "index_col"]].astype(str)
combine_df["new_latitude"] =combine_df["latitude"]+combine_df["index_col"]
combine_df

C:\Users\kelly\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\kelly\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,address,latitude,longitude,city,state,zip,Store,index_col,new_latitude
0,2400 Missouri 291 MO 64057,39.075001,-94.379703,Independence,MO,64055,Dunkin',0,39.0750010
0,3996 Jeffco Blvd MO 63010,38.40569,-90.376802,Arnold,MO,63010,McDonald's,0,38.405690
1,130 Arnold Crossroads Ctr MO 63010,38.445494,-90.374454,Arnold,MO,63010,McDonald's,1,38.4454941
2,2201 Michigan Ave MO 63010,38.43704,-90.377478,Arnold,MO,63010,McDonald's,2,38.437042
3,2005 Richardson Rd MO 63010,38.418379,-90.390784,Arnold,MO,63010,McDonald's,3,38.4183793
...,...,...,...,...,...,...,...,...,...
78,S Jefferson and Walnut,38.629806,-90.216753,St. Louis,MO,63103,Starbucks,78,38.62980678
79,Marriott St. Louis Grand,38.630655,-90.191852,St. Louis,MO,63101,Starbucks,79,38.63065579
80,Ballpark Village,38.624702,-90.193569,St. Louis,MO,63102,Starbucks,80,38.62470280
81,St. Louis Hilton at the Ballpark,38.625172,-90.190579,St. Louis,MO,63102,Starbucks,81,38.62517281


In [8]:
combine_df = combine_df.drop(['latitude'],axis=1)
combine_df = combine_df.rename(columns={"new_latitude":"latitude"}).drop(['index_col'],axis=1)
combine_df



,address,longitude,city,state,zip,Store,latitude
0,2400 Missouri 291 MO 64057,-94.379703,Independence,MO,64055,Dunkin',39.0750010
0,3996 Jeffco Blvd MO 63010,-90.376802,Arnold,MO,63010,McDonald's,38.405690
1,130 Arnold Crossroads Ctr MO 63010,-90.374454,Arnold,MO,63010,McDonald's,38.4454941
2,2201 Michigan Ave MO 63010,-90.377478,Arnold,MO,63010,McDonald's,38.437042
3,2005 Richardson Rd MO 63010,-90.390784,Arnold,MO,63010,McDonald's,38.4183793
...,...,...,...,...,...,...,...
78,S Jefferson and Walnut,-90.216753,St. Louis,MO,63103,Starbucks,38.62980678
79,Marriott St. Louis Grand,-90.191852,St. Louis,MO,63101,Starbucks,38.63065579
80,Ballpark Village,-90.193569,St. Louis,MO,63102,Starbucks,38.62470280
81,St. Louis Hilton at the Ballpark,-90.190579,St. Louis,MO,63102,Starbucks,38.62517281


In [9]:
# Push the remade DataFrame to a new CSV file
combine_df.to_csv("coffee_clean.csv",
                  encoding="utf-8", index=False, header=True)

In [10]:
check = pd.read_csv("coffee_clean.csv")
check

,address,longitude,city,state,zip,Store,latitude
0,2400 Missouri 291 MO 64057,-94.379703,Independence,MO,64055,Dunkin',39.075001
1,3996 Jeffco Blvd MO 63010,-90.376802,Arnold,MO,63010,McDonald's,38.405690
2,130 Arnold Crossroads Ctr MO 63010,-90.374454,Arnold,MO,63010,McDonald's,38.445494
3,2201 Michigan Ave MO 63010,-90.377478,Arnold,MO,63010,McDonald's,38.437042
4,2005 Richardson Rd MO 63010,-90.390784,Arnold,MO,63010,McDonald's,38.418379
...,...,...,...,...,...,...,...
313,S Jefferson and Walnut,-90.216753,St. Louis,MO,63103,Starbucks,38.629807
314,Marriott St. Louis Grand,-90.191852,St. Louis,MO,63101,Starbucks,38.630656
315,Ballpark Village,-90.193569,St. Louis,MO,63102,Starbucks,38.624703
316,St. Louis Hilton at the Ballpark,-90.190579,St. Louis,MO,63102,Starbucks,38.625173
